## Computing Gromov's delta on simulated data


In [143]:
# Torch device management
import torch

if torch.cuda.is_available():
    torch.cuda.set_device(0)
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")


Using device: cuda


In [154]:
import networkx as nx
import numpy as np

from delta import hypdelta
from hyperbolicity import sample_points as sp

In [149]:
N_ITERATIONS = 100

### Compute delta-hyperbolicity of points on a hypersphere

In [153]:
dim = 2
n_points = 500
curvatures = [1, 2, 3]
deltas = []

for _ in range(N_ITERATIONS):

    for K in curvatures:
        points, distance_mat = sp.sample_points_on_manifold(curvature=K, dim=dim, n_points=n_points)
    
        delta = hypdelta(distance_mat, device="cpu", strategy="naive")
        deltas.append(delta)

print(f"The average delta hyperbolicity of randomly sampled points on spherical manifolds is: {np.mean(deltas)}")

The average delta hyperbolicity of randomly sampled points on spherical manifolds is: 0.9896971008577237


### Compute delta-hyperbolicity of Euclidean Space

In [140]:
from scipy.spatial import distance_matrix
dim = 2
n_points = 300
K=0

points, distance_mat = sp.sample_points_on_manifold(curvature=K, dim=dim, n_points=n_points)

delta = hypdelta(distance_mat, device="cpu", strategy="naive")

print(f"The computed delta hyperbolicity of randomly sampled points at curvature {K} is: {delta}")

The computed delta hyperbolicity of randomly sampled points at curvature 0 is: 0.5185156364344387


In [61]:
from scipy.spatial import distance_matrix
dim = 2
n_points = 300
K=0

deltas = []

for _ in range(N_ITERATIONS):
    points = sp.sample_points_on_manifold(curvature=K, dim=dim, n_points=n_points)
    
    points = points.detach().numpy()
    distance_mat = distance_matrix(points, points)#m_h.dist(points[:, None], points[None, :])
    #distance_mat = distance_mat.detach().numpy()
    delta = hypdelta(distance_mat, device="cpu", strategy="naive")
    deltas.append(delta)

print(f"The computed delta hyperbolicity of randomly sampled points at curvature {K} is: {np.mean(deltas)}")

The computed delta hyperbolicity of randomly sampled points at curvature 0 is: 0.5125611688650199


In [138]:
from scipy.spatial import distance_matrix

dim = 2
n_points = 300
K=0
points, distance_mat1 = sp.sample_points_on_manifold(curvature=K, dim=dim, n_points=n_points)


points = points.detach().numpy()
distance_mat2 = distance_matrix(points, points)

delta1 = hypdelta(distance_mat1, device="cpu", strategy="naive")
delta2 = hypdelta(distance_mat2, device="cpu", strategy="naive")

print(delta1, delta2)

0.5471001347339421 0.5471001340765046


### Compute delta-hyperbolicity of discrete spaces [tree]



In [155]:
def get_distance_matrix(G):
    """
    Calculate the distance matrix for a given graph G.

    Parameters:
    G (networkx.Graph): The input graph.

    Returns:
    numpy.ndarray: A distance matrix where the element at [i, j] is the shortest path distance between nodes i and j.
    """
    # Get the number of nodes
    num_nodes = G.number_of_nodes()
    
    # Initialize the distance matrix with infinity
    distance_matrix = np.full((num_nodes, num_nodes), np.inf)
    
    # Calculate shortest paths
    shortest_paths = dict(nx.all_pairs_shortest_path_length(G))
    
    # Fill the distance matrix
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i in shortest_paths and j in shortest_paths[i]:
                distance_matrix[i, j] = shortest_paths[i][j]
            elif i == j:
                distance_matrix[i, j] = 0
    
    return distance_matrix


N_NODES = 100


deltas = []
for i in range(N_ITERATIONS):
    tree_graph = nx.random_tree(n=N_NODES)
    distance_mat = get_distance_matrix(tree_graph)
    delta = hypdelta(distance_mat, device="cpu", strategy="naive")
    deltas.append(delta)

print(f"The computed delta hyperbolicity of random trees is: {np.mean(deltas)}")

The computed delta hyperbolicity of random trees is: 0.0
